In [1]:
import pandas as pd
df=pd.read_csv(r'C:\Users\hp\Desktop\final_data.csv')
print(df.head())
df1=df[['State code','Institute','population','SUSTAINIBILITY SCORE','CA','roi','SDG Index Score']]
df1 = df1.dropna()
# from sklearn.model_selection import train_test_split
#  SDG Index Score
# X = df1.drop('', axis=1)[:2000]
# y = df1['SDG Index Score'][:2000]
# X
# X_train, X_test, y_train, y_test = train_test_split(X, df1['SDG Index Score'][:2000], test_size=0.1, random_state=42,stratify=['Institute'])

C:\Users\hp\AppData\Local\Temp\ipykernel_24832\2354499055.py:2: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'C:\Users\hp\Desktop\final_data.csv')


   DG12  DG11  SUSTAINIBILITY SCORE State code   DG7    Institute     roi  \
0  94.7  25.2                  47.0        RAJ  21.0  BITS Pilani  17.850   
1  94.5  25.2                  47.3        RAJ  22.3  BITS Pilani  17.850   
2  94.1  25.6                  47.1        RAJ  21.5  BITS Pilani  17.700   
3  94.4  26.1                  60.2        RAJ   NaN  BITS Pilani  17.775   
4  94.8  26.5                  49.8        RAJ  28.0  BITS Pilani  17.850   

     CA  population  SDG Index Score  ...  n_sdg16_weaponsexp  n_sdg16_rsf  \
0  12.0  19669913.0             36.0  ...               100.0         47.2   
1  12.0  19416052.0             36.3  ...               100.0         47.2   
2  12.0  19961213.0             36.3  ...               100.0         47.2   
3  12.0  22039300.0             36.7  ...               100.0         47.2   
4  12.0  23250960.0             37.1  ...               100.0         47.2   

   n_sdg16_justice  n_sdg16_admin  n_sdg16_exprop  n_sdg17_govex  n_

In [2]:
from sklearn.model_selection import train_test_split

# X = df1.drop('SDG Index Score', axis=1)[:2000]
# y = df1['SDG Index Score'][:2000]
# X
# X_train, X_test, y_train, y_test = train_test_split(X, df1['SDG Index Score'][:2000], test_size=0.1, random_state=42,stratify=['Institute'])

In [3]:
df1 = df1.sample(frac=1).reset_index(drop=True) # shuffle
df1.reset_index(inplace=True, drop=True)
df1['SDG Index Score'] = df1['SDG Index Score'] # add back 'Utterance'

max1 = df1['SUSTAINIBILITY SCORE'].loc[df1['SUSTAINIBILITY SCORE'].idxmax()]
df1['SUSTAINIBILITY SCORE']=df1['SUSTAINIBILITY SCORE']/max1

max2 = df1['CA'].loc[df1['CA'].idxmax()]
df1['CA']=df1['CA']/max2

max3 = df1['roi'].loc[df1['roi'].idxmax()]
df1['roi']=df1['roi']/max3

max3 = df1['population'].loc[df1['population'].idxmax()]
df1['population']=df1['population']/max3

max3 = df1['SDG Index Score'].loc[df1['SDG Index Score'].idxmax()]
df1['SDG Index Score']=df1['SDG Index Score']/max3







In [16]:
max3

86.8

In [4]:
df1

,State code,Institute,population,SUSTAINIBILITY SCORE,CA,roi,SDG Index Score
0,BIH,BIT Mesra Patna Campus,0.003618,0.736064,0.22,0.589499,0.653226
1,MP,Rabindranath tagore university Bhopal,0.009410,0.822526,0.41,0.662159,0.729263
2,TMU,VIT Vellore,0.002002,0.473265,0.17,0.547070,0.495392
3,DEL,National Law University,0.010445,0.662116,0.24,0.529833,0.551843
4,GUJ,NIFT Gandhinagar,0.007375,0.839590,0.70,0.842482,0.906682
...,...,...,...,...,...,...,...
2393,MP,Sage University Indore,0.000644,0.914676,0.48,0.559533,0.841014
2394,DEL,IIT Delhi,0.001736,0.779295,0.62,0.573323,0.710829
2395,MP,Rabindranath tagore university Bhopal,0.010253,0.833902,0.50,0.666136,0.756912
2396,TMU,SRM Chennai,0.002136,0.780432,0.64,0.765579,0.852535


In [5]:
import sys
import tqdm
import random
import numpy as np
import collections
import matplotlib.pyplot as plt

from copy import copy
from collections import defaultdict

# Local imports

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Seed random numbers
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = True
from tqdm.notebook import trange, tqdm


In [7]:
df1

,State code,Institute,population,SUSTAINIBILITY SCORE,CA,roi,SDG Index Score
0,BIH,BIT Mesra Patna Campus,0.003618,0.736064,0.22,0.589499,0.653226
1,MP,Rabindranath tagore university Bhopal,0.009410,0.822526,0.41,0.662159,0.729263
2,TMU,VIT Vellore,0.002002,0.473265,0.17,0.547070,0.495392
3,DEL,National Law University,0.010445,0.662116,0.24,0.529833,0.551843
4,GUJ,NIFT Gandhinagar,0.007375,0.839590,0.70,0.842482,0.906682
...,...,...,...,...,...,...,...
2393,MP,Sage University Indore,0.000644,0.914676,0.48,0.559533,0.841014
2394,DEL,IIT Delhi,0.001736,0.779295,0.62,0.573323,0.710829
2395,MP,Rabindranath tagore university Bhopal,0.010253,0.833902,0.50,0.666136,0.756912
2396,TMU,SRM Chennai,0.002136,0.780432,0.64,0.765579,0.852535


In [8]:
# Define data class
class TabularData(Dataset):
    def __init__(self, ids):
        y = df1['SDG Index Score'].tolist()
        
        X = df1.drop('SDG Index Score',axis=1) #without norm
        vals = X['Institute'].unique()
        mapping = {val:i for i,val in enumerate(vals)}
        X['Institute'] = [mapping[i] for i in X['Institute'].tolist()]
        vals = X['State code'].unique()
        mapping = {val:i for i,val in enumerate(vals)}
        X['State code'] = [mapping[i] for i in X['State code'].tolist()]
        X['roi'] = X['roi'].astype(int)
        X = X.drop(['Institute','State code'],axis=1)
        X = X.values.tolist()


        self.X = np.array(X).astype('float32')
        print(self.X.shape)

        # print(self.X.view(2398))
        self.y = np.array(y).astype('float32').reshape(len(y),1)
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [9]:
# for X, y in dloader['train']:
#     print(y.shape)
#     break


In [10]:
df1_val = df1[:200]
df1_test = df1[200:400]
df1_train = df1[400:]
#a lot to do as data study is not done till now

dset, dloader = {}, {}
splits = ['train','test','val']
visits = {"train":df1_train,"test":df1_test,"val":df1_val}
for split in splits:
    dset[split] = TabularData(visits[split])
    dloader[split] = DataLoader(dset[split], batch_size=256, shuffle=split == 'train')

(2398, 4)
(2398, 4)
(2398, 4)


In [11]:
# Define model
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # self.fc1 = nn.Linear(6, 6)
        # self.fc2 = nn.Linear(6, 6)
        self.fc3 = nn.Linear(4, 1)

        

        # self.fc = nn.Sequential(nn.Linear(2, 5), nn.ReLU(), nn.Linear(5, 1))
    
    def forward(self, X):
        # X = self.fc1(X)
        # X = self.fc2(X)
        X = F.sigmoid(self.fc3(X))
        return X



In [12]:
# create model, optimizer, criteria
model = MLP()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
crit = nn.MSELoss()
# crit = nn.L1Loss()

In [ ]:
# train for several epochs

N_EPOCHS = 10000
train_mae = []
val_mae = []
pbar = trange(N_EPOCHS, desc=f'Epoch: {0:4d}, Train MAE: {np.inf:.4f}, Val MAE: {np.inf:.4f}')
for ep in pbar:
    train_errors, val_errors = [], []

    model.train()
    for X, y in dloader['train']:
        optimizer.zero_grad()
        yhat = model(X)
        loss = crit(yhat, y)
        loss.backward()
        optimizer.step()
        train_errors.append((yhat - y).abs())
    
    with torch.no_grad():
        model.eval()
        for X, y in dloader['val']:
            yhat = model(X)
            val_errors.append((yhat - y).abs())

    train_mae.append(torch.cat(train_errors).mean().item())
    val_mae.append(torch.cat(val_errors).mean().item())
    pbar.set_description(f'Epoch: {ep:4d}, TRN MAE: {train_mae[-1]:.4f}, VAL MAE: {val_mae[-1]:.4f}')


# Print final train and val MAE
print(f'Final Train MAE: {1000 * train_mae[-1] : .1f} g, Val MAE: {1000 * val_mae[-1] : .1f} ')

best_train_idx = train_mae.index(min(train_mae))
print(f'Best Train MAE: {1000 * train_mae[best_train_idx] : .1f} g, Val MAE: {1000 * val_mae[best_train_idx] : .1f} g, Epoch: {best_train_idx + 1}')

Epoch:    0, Train MAE: inf, Val MAE: inf:   0%|          | 0/10000 [00:00<?, ?it/s]

In [13]:
#saving and loding pytorch model
PATH = 'tabular.pth'
torch.save(model.state_dict(), PATH)


In [14]:
#inference code

In [28]:
# Define model
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc3 = nn.Linear(4, 1)
        # self.fc = nn.Sequential(nn.Linear(2, 5), nn.ReLU(), nn.Linear(5, 1))
    
    def forward(self, X):
        # X = self.fc1(X)
        # X = self.fc2(X)
        X = F.sigmoid(self.fc3(X))
        return X


model = MLP()
model.load_state_dict(torch.load(PATH)) #use this 

def inf(x):
    with torch.no_grad():
        model.eval()
        yhat = model(x)
        yhat=yhat.item()
        yhat=yhat*86.8
        return yhat
    
x = [32071600.00,53.70,15.00,18.225]
##normlize
x[0] = x[0]/1425925386.00
x[1] = x[1]/87.90
x[2] = x[2]/100.00
x[3] = x[3]/94.275

#convert them into pytorch tensor
b=torch.tensor(x)
b=b.to(torch.float32)
print(inf(b))
#give them to the model
#get the output

39.899548053741455
